In [134]:
import pandas as pd

In [135]:
csv_1 = pd.read_csv('Consensus2.csv', index_col=0, sep='|')

In [136]:
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The commonly agreed-upon answer is ""contrad..."
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The commonly agreed-upon answer is ""neutral..."
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The commonly agreed-upon answer is ""entailm..."
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The commonly agreed-upon answer is ""entailm..."
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The commonly agreed-upon answer is ""neutral..."
...,...,...,...
3895,A man reading a book on the sidewalk with just...,An old man sits outside while he wears a blue ...,"6. The commonly agreed-upon answer is ""entailm..."
3896,A couple fishes from a rocky bank.,Two fish are swimming.,"7. The commonly agreed-upon answer is ""neutral..."
3897,A cashier with a black goodies sweatshirt is c...,A cashier counting change.,"8. The commonly agreed-upon answer is ""entailm..."
3898,A young female poses in blue jeans and a blue ...,The model is in a photo shoot with a new Ford ...,"9. The commonly agreed-upon answer is ""entailm..."


In [137]:
csv_2 = pd.read_csv('Consensus.csv', index_col=0, sep='|')
csv_2

,premise,hypothesis,rationales
3900,An ethnic woman wearing a purple apron is sitt...,The woman is alone in her bathroom.,"1. The answer is ""contradiction"". The premise ..."
3901,Janitor cleans off diving board with broom at ...,A janitor cleans a diving board.,"2. The answer is ""neutral"". The premise states..."
3902,A girl wearing a yellow shirt playing with a b...,The busted pinata had candy in it.,"3. The answer is ""entailment"". The premise des..."
3903,A crowd is busy looking to buy something.,A single man is looking to buy something.,"4. The answer is ""neutral"". The premise states..."
3904,A girl wearing glasses and a brown plaid long-...,The girl is sitting on a rug.,"5. The answer is ""neutral"". The premise descri..."
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,8. The commonly agreed-upon answer to the prem...
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. The commonly agreed-upon answer to the prem...
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. The commonly agreed-upon answer to the pre...
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,Answer: Neutral Justification: Both the premis...


In [138]:
csv_1 = pd.concat([csv_1, csv_2])
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The commonly agreed-upon answer is ""contrad..."
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The commonly agreed-upon answer is ""neutral..."
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The commonly agreed-upon answer is ""entailm..."
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The commonly agreed-upon answer is ""entailm..."
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The commonly agreed-upon answer is ""neutral..."
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,8. The commonly agreed-upon answer to the prem...
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. The commonly agreed-upon answer to the prem...
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. The commonly agreed-upon answer to the pre...
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,Answer: Neutral Justification: Both the premis...


In [139]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [140]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
What is the commonly agreed-upon answer to the premise '%s' and hypothesis %s with options "entailment", "neutral", "contradiction"? Justify your answer based on general knowledge. \n
'''

In [141]:
error = csv_1[csv_1.rationales.isna()]

In [142]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [143]:
error

,premise,hypothesis,rationales,prompt


In [144]:
print(error.prompt[0:].values)

[]


In [145]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [146]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [147]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [114]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [148]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [149]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [150]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [151]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [152]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [153]:
paper.shape

(5592, 6)

In [154]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The commonly agreed-upon answer is ""contrad...","1. the commonly agreed-upon answer is ""contrad...","{'contradiction': 39, 'contradictory': 280}",contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The commonly agreed-upon answer is ""neutral...","2. the commonly agreed-upon answer is ""neutral...",{'neutral': 39},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The commonly agreed-upon answer is ""entailm...","3. the commonly agreed-upon answer is ""entailm...",{'entailment': 39},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The commonly agreed-upon answer is ""entailm...","4. the commonly agreed-upon answer is ""entailm...",{'entailment': 39},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The commonly agreed-upon answer is ""neutral...","5. the commonly agreed-upon answer is ""neutral...",{'neutral': 39},neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. The commonly agreed-upon answer to the prem...,8. the commonly agreed-upon answer to the prem...,{'neutral': 177},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. The commonly agreed-upon answer to the prem...,9. the commonly agreed-upon answer to the prem...,{'neutral': 222},neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. The commonly agreed-upon answer to the pre...,10. the commonly agreed-upon answer to the pre...,{'entailment': 267},entailment


In [155]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'

In [156]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,"1. The commonly agreed-upon answer is ""contrad...","1. the commonly agreed-upon answer is ""contrad...","{'contradiction': 39, 'contradictory': 280}",contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,"2. The commonly agreed-upon answer is ""neutral...","2. the commonly agreed-upon answer is ""neutral...",{'neutral': 39},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,"3. The commonly agreed-upon answer is ""entailm...","3. the commonly agreed-upon answer is ""entailm...",{'entailment': 39},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,"4. The commonly agreed-upon answer is ""entailm...","4. the commonly agreed-upon answer is ""entailm...",{'entailment': 39},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,"5. The commonly agreed-upon answer is ""neutral...","5. the commonly agreed-upon answer is ""neutral...",{'neutral': 39},neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. The commonly agreed-upon answer to the prem...,8. the commonly agreed-upon answer to the prem...,{'neutral': 177},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. The commonly agreed-upon answer to the prem...,9. the commonly agreed-upon answer to the prem...,{'neutral': 222},neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. The commonly agreed-upon answer to the pre...,10. the commonly agreed-upon answer to the pre...,{'entailment': 267},entailment


In [157]:
paper['my_label'] = total_df['LLM_answer']

In [158]:
sum(paper.label == paper.my_label) / len(paper)

0.630722460658083

In [159]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [160]:
total_df.to_csv('consensus - full.csv')

In [195]:
csv_1.to_csv('neutral - full.csv', index=False)

In [162]:
total_df.index.value_counts()

A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.                                                                     1
a man taking a pizza out of an oven</s>A woman is sitting in the oven.                                                                                                                                                                  1
Side view of a mountaineer walking into strong headwinds.</s>A man outside.                                                                                                                                                             1
A boy jumping off of a diving board while a girl watches.</s>A boy is near water.                                                                                                                                                       1
A back view of a man driving a tractor and signaling with two ch